## Toronto Neighbourhood

#### Import Libraries

In [27]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::jupyter_console==6.0.0=py36_0
  - defaults/osx-64::spyder-kernels==0.3.0=py36_0
  - defaults/osx-64::jupyterlab_server==0.2.0=py36_0
  - defaults/osx-64::wheel==0.32.3=py36_0
  - defaults/osx-64::descartes==1.1.0=py36_0
  - defaults/osx-64::flask==1.0.2=py36_1
  - defaults/osx-64::networkx==2.2=py36_1
  - conda-forge/label/cf201901/osx-64::tensorflow==1.10.0=py36_0
  - defaults/osx-64::sphinx==1.8.2=py36_0
  - defaults/osx-64::pytest-remotedata==0.3.1=py36_0
  - defaults/osx-64::widgetsnbextension==3.4.2=py36_0
  - defaults/osx-64::pytest-openfiles==0.3.2=py36_0
  - defaults/osx-64::seaborn==0.9.0=py36_0
  - defaults/osx-64::jinja2==2.10=py36_0
  - defaults/osx-64::ipython==7.2.0=py36h39e3cac_0
  - defaults/osx-64::pytest-astropy==0.3.0=py36_0
  - defaults/osx-64::grpcio==1.16.1=py36h044775b_1
  - defaults/osx-64::a

### URL read the table in DataFrame

In [28]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, index_col=0)[0]
df.reset_index(inplace=True)
df.head(2)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned


### Clean the data frame

#### Removing the Not Assigned Borough
Drop the rows with Not Assigned Borough

In [29]:
# Clean the unassigned Borough
cleandf = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)

#### Concatenate the Neighbourhood with same Post code
Check if the join occured correctly or not

In [30]:
combdf = cleandf.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

Check if the join occured correctly or not .
Orgininal data with 2 different Neighbourhood with same Postcode

In [31]:
cleandf[cleandf['Postcode'] =='M5A']

,Postcode,Borough,Neighbourhood
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park


In [32]:
# corrected DataFrame
combdf[combdf['Postcode'] =='M5A']

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"


#### Not Assigned Neighbourhood will have same value as their Borough

In [33]:
# Not assigned Neighbourhood
combdf.loc[combdf.Neighbourhood == 'Not assigned','Neighbourhood'] = combdf['Borough']

In [34]:
#Check the not assigned Neighbourhood
print("Orginal Data with Not assigned Neighbourhood:\n ")
print(cleandf[cleandf['Neighbourhood'] =='Not assigned'])
print("\nNew Data with Not assigned Neighbourhood:\n ")
print(combdf[combdf['Postcode'] =='M7A'])
print("\nCheck Not assigned Neighbourhood:\n ")
print(combdf[combdf['Neighbourhood'] =='Not assigned'])

Orginal Data with Not assigned Neighbourhood:
 
  Postcode       Borough Neighbourhood
6      M7A  Queen's Park  Not assigned

New Data with Not assigned Neighbourhood:
 
   Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Queen's Park

Check Not assigned Neighbourhood:
 
Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


#### Check the shape of final DataFrame

In [35]:
combdf.shape

(103, 3)

### Geocode
Add latitudes and longitudes to the postal code 

In [36]:
geocode = pd.read_csv("Geospatial_Coordinates.csv")
geocode.head(2)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


In [37]:
combdf['Latitude']= combdf['Postcode'].map(geocode.set_index('Postal Code')['Latitude'])
combdf['Longitude']= combdf['Postcode'].map(geocode.set_index('Postal Code')['Longitude'])


In [38]:
combdf.head(2)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497


### Explore and cluster the neighborhoods in Toronto

In [42]:
# find all the borough with Toronto
tornoDf = combdf[combdf['Borough'].str.contains("Toronto")]


### Create Toronto Map
Using Folium library create map of Toronto with neighborhoods superimposed on top.

In [44]:
# create map of Toronto using latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="Nominatim_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [47]:
# create map of Toronto using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(tornoDf['Latitude'], tornoDf['Longitude'], tornoDf['Borough']):
    label = '{}'.format( borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2. Explore Borough  in Toronto

In [61]:
tornoDf.reset_index(inplace=True)

In [62]:
neighborhood_latitude = tornoDf.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tornoDf.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = tornoDf.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of East Toronto are 43.67635739999999, -79.2930312.


In [63]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [65]:
# type your answer here

torno_venues = getNearbyVenues(names=tornoDf['Borough'],
                                   latitudes=tornoDf['Latitude'],
                                   longitudes=tornoDf['Longitude']
                                  )


East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
East Toronto


In [66]:
print(torno_venues.shape)
torno_venues.head()

(1705, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,East Toronto,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [67]:
print('There are {} uniques categories.'.format(len(torno_venues['Venue Category'].unique())))

There are 231 uniques categories.


In [68]:
# one hot encoding
torno_onehot = pd.get_dummies(torno_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torno_onehot['Neighborhood'] = torno_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torno_onehot.columns[-1]] + list(torno_onehot.columns[:-1])
torno_onehot = torno_onehot[fixed_columns]

torno_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [69]:
torno_grouped = torno_onehot.groupby('Neighborhood').mean().reset_index()

### Cluster Analysis

In [84]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torno_grouped['Neighborhood']

for ind in np.arange(torno_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torno_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Pizza Place,Sandwich Place,Café,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Restaurant,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Park,Gastropub
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Ice Cream Shop,Pizza Place,Bakery,American Restaurant,Pub
3,West Toronto,Bar,Café,Coffee Shop,Bakery,Italian Restaurant,Pizza Place,Restaurant,Breakfast Spot,Music Venue,Asian Restaurant


In [89]:
# set number of clusters
kclusters = 3

torno_grouped_clustering = torno_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torno_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 1], dtype=int32)

In [90]:
tornoDf.rename(columns={"Borough":"Neighborhood"}, inplace=True)

/Users/vinitachaudhary/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [91]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torno_merged = tornoDf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torno_merged = torno_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

torno_merged.head() # check the last columns!

,index,Postcode,Neighborhood,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Ice Cream Shop,Pizza Place,Bakery,American Restaurant,Pub
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Ice Cream Shop,Pizza Place,Bakery,American Restaurant,Pub
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Ice Cream Shop,Pizza Place,Bakery,American Restaurant,Pub
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Ice Cream Shop,Pizza Place,Bakery,American Restaurant,Pub
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Pizza Place,Sandwich Place,Café,Park,Dessert Shop,Sushi Restaurant,Clothing Store,Restaurant,Pub


In [92]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torno_merged['Latitude'], torno_merged['Longitude'], torno_merged['Neighborhood'], torno_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters